#### 肾脏无回声+肾脏囊肿 订单描述

In [1]:
import pymysql
import pandas as pd
import numpy as np

In [24]:
conn=pymysql.connect(host='mysql-backup.shanzhen.me',
                     user='readonly_medicine_liuqq',
                     password="r3g1^QWgV8GeYJw4",
                     database='asgard_dataplatform',
                     port= 3001,
                     charset='utf8')

In [1]:
id_list = ["OBDIM0006Tag0005","OBDIM0006Tag0028"]

In [6]:
tag_df = pd.DataFrame()

for items in id_list:
    id_str = "'"+ items + "'"
    tagsql = 'SELECT ORDER_CODE,TAG_ID \
    FROM t66_apollo_offline_analysis_tag \
    WHERE SOURCE_TYPE = 1 \
    AND SZ_STATUS = 1 AND TAG_ID IN ('+id_str+') '
    tag = pd.read_sql(tagsql,con=conn)
    tag_df = pd.concat([tag_df,tag],axis = 0) 
    
tag_df    

,ORDER_CODE,TAG_ID
0,000012347841,OBDIM0006Tag0005
1,000017779833,OBDIM0006Tag0005
2,000027055939,OBDIM0006Tag0005
3,000034935327,OBDIM0006Tag0005
4,000035346462,OBDIM0006Tag0005
...,...,...
57729,385406074910,OBDIM0006Tag0028
57730,385485134085,OBDIM0006Tag0028
57731,385568674544,OBDIM0006Tag0028
57732,385624408650,OBDIM0006Tag0028


In [8]:
tag_df1 = tag_df.drop_duplicates(subset = 'ORDER_CODE')
tag_df1

,ORDER_CODE,TAG_ID
0,000012347841,OBDIM0006Tag0005
1,000017779833,OBDIM0006Tag0005
2,000027055939,OBDIM0006Tag0005
3,000034935327,OBDIM0006Tag0005
4,000035346462,OBDIM0006Tag0005
...,...,...
57600,381300153204,OBDIM0006Tag0028
57635,382514131828,OBDIM0006Tag0028
57658,382965087472,OBDIM0006Tag0028
57710,384740983563,OBDIM0006Tag0028


In [10]:
tag_df1.ORDER_CODE

0        000012347841
1        000017779833
2        000027055939
3        000034935327
4        000035346462
             ...     
57600    381300153204
57635    382514131828
57658    382965087472
57710    384740983563
57733    385663589304
Name: ORDER_CODE, Length: 60249, dtype: object

In [13]:
Recode_df = pd.DataFrame()

for items in tag_df1.ORDER_CODE:
    ORDER_CODE_str = "'"+ items + "'"
    
    sqlstr = "SELECT DISTINCT RR.ORDER_CODE,RR2.RAW_NAME,RR.RAW_NAME,RR.RAW_RESULT AS '三级项结果',RR2.RAW_RESULT AS '小结',DR.SD_CODE \
    FROM T66_ATHENA_BASIC_RAW_RECORD AS RR LEFT JOIN T66_ATHENA_BASIC_DESCRIPTIVE_RECORD AS DR ON RR.ORDER_CODE = DR.ORDER_CODE \
    AND RR.RECORD_CODE = DR.RECORD_CODE AND DR.SD_CODE IN ('S0000005D','S0000006D','S0000007D') \
    AND RR.ORDER_CODE IN (" + ORDER_CODE_str + ")AND DR.SZ_STATUS = 1 \
    LEFT JOIN T66_ATHENA_BASIC_RAW_RECORD AS RR2 ON RR.ORDER_CODE = RR2.ORDER_CODE \
    AND RR.PARENT_RECORD_CODE = RR2.RECORD_CODE \
    WHERE \
    DR.ID IS NOT NULL \
    AND RR.SZ_STATUS = 1 \
    AND DR.SZ_STATUS = 1 \
    AND RR2.SZ_STATUS = 1 " 

    Recode = pd.read_sql(sqlstr,con=conn)
    Recode_df = pd.concat([Recode_df,Recode],axis = 0) 

Recode_df

,ORDER_CODE,RAW_NAME,RAW_NAME,三级项结果,小结,SD_CODE
0,000012347841,腹部超声,左肾,左肾形态大小正常，包膜光整，实质回声均匀，肾内可见一个无回声区，大小约33mmx23mm，后...,肝:肝脏回声粗糙\r\n肝囊肿\r\n左肾:左肾囊肿\r\n左肾结晶\r\n右肾:右肾结晶\...,S0000006D
1,000012347841,腹部超声,右肾,右肾形态大小正常，包膜光整，实质回声均匀，肾内可见点状强回声，后无明显声影。集合系统未见分离。,肝:肝脏回声粗糙\r\n肝囊肿\r\n左肾:左肾囊肿\r\n左肾结晶\r\n右肾:右肾结晶\...,S0000007D
0,000017779833,肝胆胰脾肾超声,左肾,左肾大小、形态正常，包膜光滑，肾实质回声均匀，集合系统未见明显分离。,右肾:右肾囊肿（多发）\r\n肝、胆、胰、脾、左肾未发现明显异常,S0000006D
1,000017779833,肝胆胰脾肾超声,右肾,右肾大小、形态可，于肾内探及多个无回声，较大者约2.2×1.6CM，边界尚清。集合系统未见明...,右肾:右肾囊肿（多发）\r\n肝、胆、胰、脾、左肾未发现明显异常,S0000007D
0,000027055939,腹部超声,左肾,左肾形态大小正常，包膜光整，实质回声均匀，肾内可见多个点状强回声，后无明显声影。肾内可见一个...,肝:脂肪肝（轻度）胆:胆囊壁欠光滑左肾:左肾结晶左肾囊肿右肾:右肾结晶胰、脾未发现明显异常,S0000006D
...,...,...,...,...,...,...
1,382514131828,腹部超声,右肾,右肾大小、形态正常，包膜光滑，肾实质回声均匀，集合系统未见明显分离。,肝:肝囊肿 胆、胰、脾、右肾、左肾未发现明显异常,S0000007D
0,382965087472,彩超室,双肾,双肾大小形态正常，实质与集合系统分界清晰，右肾可见一强回声区，直径约7mm，后伴声影；左肾可...,肝内强回声区，考虑：钙化\r\n右肾强回声区，考虑：结石\r\n左肾极低回声区，囊肿可能？\...,S0000005D
0,384740983563,腹部超声,左肾,左肾形态大小正常，包膜光整，实质回声均匀。集合系统光点回声增强。,肝:脂肪肝趋势 肝囊肿 胆、胰、脾、左肾、右肾未发现明显异常,S0000006D
1,384740983563,腹部超声,右肾,右肾形态大小正常，包膜光整，实质回声均匀。集合系统光点回声增强。,肝:脂肪肝趋势 肝囊肿 胆、胰、脾、左肾、右肾未发现明显异常,S0000007D


In [96]:
Recode_df1 = Recode_df.drop_duplicates(subset = ['ORDER_CODE','三级项结果','小结'])
Recode_df1

,ORDER_CODE,RAW_NAME,RAW_NAME,三级项结果,小结,SD_CODE
0,000012347841,腹部超声,左肾,左肾形态大小正常，包膜光整，实质回声均匀，肾内可见一个无回声区，大小约33mmx23mm，后...,肝:肝脏回声粗糙\r\n肝囊肿\r\n左肾:左肾囊肿\r\n左肾结晶\r\n右肾:右肾结晶\...,S0000006D
1,000012347841,腹部超声,右肾,右肾形态大小正常，包膜光整，实质回声均匀，肾内可见点状强回声，后无明显声影。集合系统未见分离。,肝:肝脏回声粗糙\r\n肝囊肿\r\n左肾:左肾囊肿\r\n左肾结晶\r\n右肾:右肾结晶\...,S0000007D
0,000017779833,肝胆胰脾肾超声,左肾,左肾大小、形态正常，包膜光滑，肾实质回声均匀，集合系统未见明显分离。,右肾:右肾囊肿（多发）\r\n肝、胆、胰、脾、左肾未发现明显异常,S0000006D
1,000017779833,肝胆胰脾肾超声,右肾,右肾大小、形态可，于肾内探及多个无回声，较大者约2.2×1.6CM，边界尚清。集合系统未见明...,右肾:右肾囊肿（多发）\r\n肝、胆、胰、脾、左肾未发现明显异常,S0000007D
0,000027055939,腹部超声,左肾,左肾形态大小正常，包膜光整，实质回声均匀，肾内可见多个点状强回声，后无明显声影。肾内可见一个...,肝:脂肪肝（轻度）胆:胆囊壁欠光滑左肾:左肾结晶左肾囊肿右肾:右肾结晶胰、脾未发现明显异常,S0000006D
...,...,...,...,...,...,...
1,382514131828,腹部超声,右肾,右肾大小、形态正常，包膜光滑，肾实质回声均匀，集合系统未见明显分离。,肝:肝囊肿 胆、胰、脾、右肾、左肾未发现明显异常,S0000007D
0,382965087472,彩超室,双肾,双肾大小形态正常，实质与集合系统分界清晰，右肾可见一强回声区，直径约7mm，后伴声影；左肾可...,肝内强回声区，考虑：钙化\r\n右肾强回声区，考虑：结石\r\n左肾极低回声区，囊肿可能？\...,S0000005D
0,384740983563,腹部超声,左肾,左肾形态大小正常，包膜光整，实质回声均匀。集合系统光点回声增强。,肝:脂肪肝趋势 肝囊肿 胆、胰、脾、左肾、右肾未发现明显异常,S0000006D
1,384740983563,腹部超声,右肾,右肾形态大小正常，包膜光整，实质回声均匀。集合系统光点回声增强。,肝:脂肪肝趋势 肝囊肿 胆、胰、脾、左肾、右肾未发现明显异常,S0000007D


In [16]:
Recode_df1.to_pickle(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\肾脏\回声内部信息提取\Recode.pkl")

In [97]:
Recode_df1 = Recode_df1.reset_index(drop = True)
Recode_df1

,ORDER_CODE,RAW_NAME,RAW_NAME,三级项结果,小结,SD_CODE
0,000012347841,腹部超声,左肾,左肾形态大小正常，包膜光整，实质回声均匀，肾内可见一个无回声区，大小约33mmx23mm，后...,肝:肝脏回声粗糙\r\n肝囊肿\r\n左肾:左肾囊肿\r\n左肾结晶\r\n右肾:右肾结晶\...,S0000006D
1,000012347841,腹部超声,右肾,右肾形态大小正常，包膜光整，实质回声均匀，肾内可见点状强回声，后无明显声影。集合系统未见分离。,肝:肝脏回声粗糙\r\n肝囊肿\r\n左肾:左肾囊肿\r\n左肾结晶\r\n右肾:右肾结晶\...,S0000007D
2,000017779833,肝胆胰脾肾超声,左肾,左肾大小、形态正常，包膜光滑，肾实质回声均匀，集合系统未见明显分离。,右肾:右肾囊肿（多发）\r\n肝、胆、胰、脾、左肾未发现明显异常,S0000006D
3,000017779833,肝胆胰脾肾超声,右肾,右肾大小、形态可，于肾内探及多个无回声，较大者约2.2×1.6CM，边界尚清。集合系统未见明...,右肾:右肾囊肿（多发）\r\n肝、胆、胰、脾、左肾未发现明显异常,S0000007D
4,000027055939,腹部超声,左肾,左肾形态大小正常，包膜光整，实质回声均匀，肾内可见多个点状强回声，后无明显声影。肾内可见一个...,肝:脂肪肝（轻度）胆:胆囊壁欠光滑左肾:左肾结晶左肾囊肿右肾:右肾结晶胰、脾未发现明显异常,S0000006D
...,...,...,...,...,...,...
85624,382514131828,腹部超声,右肾,右肾大小、形态正常，包膜光滑，肾实质回声均匀，集合系统未见明显分离。,肝:肝囊肿 胆、胰、脾、右肾、左肾未发现明显异常,S0000007D
85625,382965087472,彩超室,双肾,双肾大小形态正常，实质与集合系统分界清晰，右肾可见一强回声区，直径约7mm，后伴声影；左肾可...,肝内强回声区，考虑：钙化\r\n右肾强回声区，考虑：结石\r\n左肾极低回声区，囊肿可能？\...,S0000005D
85626,384740983563,腹部超声,左肾,左肾形态大小正常，包膜光整，实质回声均匀。集合系统光点回声增强。,肝:脂肪肝趋势 肝囊肿 胆、胰、脾、左肾、右肾未发现明显异常,S0000006D
85627,384740983563,腹部超声,右肾,右肾形态大小正常，包膜光整，实质回声均匀。集合系统光点回声增强。,肝:脂肪肝趋势 肝囊肿 胆、胰、脾、左肾、右肾未发现明显异常,S0000007D


In [17]:
location = pd.read_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\肾脏\回声内部信息提取\location.xlsx")
location

,location
0,肾
1,左肾
2,右肾
3,输尿管
4,皮质
5,髓质
6,实质
7,集合系统
8,肾盂
9,肾盏


In [24]:
location_str = ("|".join(location.location.values))
location_str

'肾|左肾|右肾|输尿管|皮质|髓质|实质|集合系统|肾盂|肾盏'

In [148]:
import re

# def extract_internal_info():
df_result = pd.DataFrame(columns=['ORDER_CODE', '文本','无回声描述','无回声外','无回声内'])
location_str = ("|".join(location.location.values))
regex_split = r"("+location_str+")"
for  index,text in enumerate (Recode_df1['三级项结果']):
    order_code = Recode_df1.ORDER_CODE[index]
    result = re.split(regex1,text )
    result_split = []
    for item in result:
        if item in location.location.values or item == "":
            continue
        else:
            result_split.append(item)
    split_list = []
    result_find_ex = []
    result_find_in = []
    for split in result_split:
        regex_find = r"((无\s?回\s?声\s?.*\s?).*(内\s?.*))"
        result_ = re.search(regex_find, split)
        if result_ != None:
            
            split_list.append(split)
            result_find_ex.append(result_.group(2))
            result_find_in.append(result_.group(3))
   
    if result_find_in :
        df = pd.DataFrame({"ORDER_CODE":order_code,"文本":text,"无回声描述":split_list,"无回声外":result_find_ex,"无回声内":result_find_in})       
    else:
        continue
    df_result = pd.concat([df_result,df],axis = 0)        
        
df_result                           

,ORDER_CODE,文本,无回声描述,无回声外,无回声内
0,000162404317,右肾形态大小正常，包膜光整，实质回声均匀，可见一个无回声区，大小约 2.5*1.6cm，内透...,回声均匀，可见一个无回声区，大小约 2.5*1.6cm，内透声差，后方回声增强。,无回声区，大小约 2.5*1.6cm，,内透声差，后方回声增强。
0,385927579832,双肾内见点状强回声。左肾内见无回声区，大小约10×8mm，其内见钙乳回声。CDFI：血流显示正常。,内见无回声区，大小约10×8mm，其内见钙乳回声。CDFI：血流显示正常。,无回声区，大小约10×8mm，其,内见钙乳回声。CDFI：血流显示正常。
0,000367011493,左肾形态大小位置正常，包膜完整，实质回声正常。右肾内见无回声区，大小约9×8mm，其内见钙乳...,内见无回声区，大小约9×8mm，其内见钙乳回声。CDFI：血流显示正常。,无回声区，大小约9×8mm，其,内见钙乳回声。CDFI：血流显示正常。
0,386255498160,双肾形态大小正常范围，包膜光滑，集合系统未见明显分离。右肾内见无回声区，大小约12x13mm...,内见无回声区，大小约12x13mm，内见强回声。CDFI：血流灌注良好。,无回声区，大小约12x13mm，,内见强回声。CDFI：血流灌注良好。
0,000609239134,左肾大小正常，实质回声均匀，肾内可见点状强回声，后方不伴声影。肾内见一大小约11mmx8mm...,内见一大小约11mmx8mm的无回声区，其内见点状强回声。,无回声区，其,内见点状强回声。
...,...,...,...,...,...
0,896685929416,左肾形态大小正常，包膜光整，实质回声均匀，肾实质与集合系统交界处可见一个无回声区，大小约13...,交界处可见一个无回声区，大小约13x13mm，壁薄界清，内透声好，后方回声增强。,无回声区，大小约13x13mm，壁薄界清，,内透声好，后方回声增强。
0,317356126084,左肾大小测值正常，实质回声均匀，集合系统未见明显分离及结石影。左肾盂内见10mmx10mm无...,盂内见10mmx10mm无回声，壁薄，囊内清。,无回声，壁薄，囊,内清。
0,342801107540,左肾形态大小正常，包膜光整，实质回声均匀，肾盂旁可见一个无回声区，大小约18.9mm×16....,盂旁可见一个无回声区，大小约18.9mm×16.7mm，内见点状强回声。另于,无回声区，大小约18.9mm×16.7mm，,内见点状强回声。另于
0,365688365854,左肾盂内见27.6*22.2mm无回声，壁薄，囊内清。,盂内见27.6*22.2mm无回声，壁薄，囊内清。,无回声，壁薄，囊,内清。


In [152]:
import re

# def extract_internal_info():
df_result = pd.DataFrame(columns=['ORDER_CODE', '文本','无回声描述','无回声外','无回声内'])
location_str = ("|".join(location.location.values))
regex_split = r"("+location_str+")"
for  index,text in enumerate (Recode_df1['三级项结果']):
    order_code = Recode_df1.ORDER_CODE[index]
    result = re.split(regex1,text )
    result_split = []
    for item in result:
        if item in location.location.values or item == "":
            continue
        else:
            result_split.append(item)
    split_list = []
    result_find_ex = []
    result_find_in = []
    for split in result_split:
        regex_find = r"((无\s?回\s?声\s?.*\s?.*)(内\s?.*))"
        result_ = re.search(regex_find, split)
        if result_ != None:
            
            split_list.append(split)
            result_find_ex.append(result_.group(2))
            result_find_in.append(result_.group(3))
   
    if result_find_in :
        df = pd.DataFrame({"ORDER_CODE":order_code,"文本":text,"无回声描述":split_list,"无回声外":result_find_ex,"无回声内":result_find_in})       
    else:
        continue
    df_result = pd.concat([df_result,df],axis = 0)        
        
df_result                           

,ORDER_CODE,文本,无回声描述,无回声外,无回声内
0,000162404317,右肾形态大小正常，包膜光整，实质回声均匀，可见一个无回声区，大小约 2.5*1.6cm，内透...,回声均匀，可见一个无回声区，大小约 2.5*1.6cm，内透声差，后方回声增强。,无回声区，大小约 2.5*1.6cm，,内透声差，后方回声增强。
0,385927579832,双肾内见点状强回声。左肾内见无回声区，大小约10×8mm，其内见钙乳回声。CDFI：血流显示正常。,内见无回声区，大小约10×8mm，其内见钙乳回声。CDFI：血流显示正常。,无回声区，大小约10×8mm，其,内见钙乳回声。CDFI：血流显示正常。
0,000367011493,左肾形态大小位置正常，包膜完整，实质回声正常。右肾内见无回声区，大小约9×8mm，其内见钙乳...,内见无回声区，大小约9×8mm，其内见钙乳回声。CDFI：血流显示正常。,无回声区，大小约9×8mm，其,内见钙乳回声。CDFI：血流显示正常。
0,386255498160,双肾形态大小正常范围，包膜光滑，集合系统未见明显分离。右肾内见无回声区，大小约12x13mm...,内见无回声区，大小约12x13mm，内见强回声。CDFI：血流灌注良好。,无回声区，大小约12x13mm，,内见强回声。CDFI：血流灌注良好。
0,000609239134,左肾大小正常，实质回声均匀，肾内可见点状强回声，后方不伴声影。肾内见一大小约11mmx8mm...,内见一大小约11mmx8mm的无回声区，其内见点状强回声。,无回声区，其,内见点状强回声。
...,...,...,...,...,...
0,896685929416,左肾形态大小正常，包膜光整，实质回声均匀，肾实质与集合系统交界处可见一个无回声区，大小约13...,交界处可见一个无回声区，大小约13x13mm，壁薄界清，内透声好，后方回声增强。,无回声区，大小约13x13mm，壁薄界清，,内透声好，后方回声增强。
0,317356126084,左肾大小测值正常，实质回声均匀，集合系统未见明显分离及结石影。左肾盂内见10mmx10mm无...,盂内见10mmx10mm无回声，壁薄，囊内清。,无回声，壁薄，囊,内清。
0,342801107540,左肾形态大小正常，包膜光整，实质回声均匀，肾盂旁可见一个无回声区，大小约18.9mm×16....,盂旁可见一个无回声区，大小约18.9mm×16.7mm，内见点状强回声。另于,无回声区，大小约18.9mm×16.7mm，,内见点状强回声。另于
0,365688365854,左肾盂内见27.6*22.2mm无回声，壁薄，囊内清。,盂内见27.6*22.2mm无回声，壁薄，囊内清。,无回声，壁薄，囊,内清。


In [153]:
df_result.to_excel(r"C:\Users\13670\Desktop\df_result1.xlsx")

In [158]:
total1_

,index,ORDER_CODE
0,0,000162404317
1,0,385927579832
2,0,000367011493
3,0,386255498160
4,0,000609239134
...,...,...
5458,80270,629334716532
5459,80601,632172803615
5460,80636,632489785799
5461,80947,373212820193


In [159]:
total1_ = df_result.ORDER_CODE.drop_duplicates().append(not_find.ORDER_CODE.drop_duplicates())
total1_ = total1_.reset_index(drop = True)
total1_[total1_.duplicated()== True]

5360    640436426406
5363    399896437277
5365    016258266721
5366    021567798526
5368    419683249503
            ...     
5457    992862835811
5458    629334716532
5460    632489785799
5461    373212820193
5462    375011614965
Name: ORDER_CODE, Length: 80, dtype: object

In [114]:
result_xd = pd.read_excel(r"C:\Users\13670\Desktop\keyishan3.xlsx")
result_xd

,Unnamed: 0,ORDER_CODE,desc,nei
0,121,584874661737,双肾形态大小正常范围，包膜光滑，集合系统未见明显分离。右肾内见无回声区，大小约80x60mm...,肾内见无回声||区，大小约80x60mm，内见分隔波，呈分叶状；右
1,129,572129765294,双肾形态大小正常范围，包膜光滑，集合系统未见明显分离。左肾见无回声，大小约8×6mm。右肾见...,肾见两枚无回声||，大小约11×10mm，之一囊壁内见点状强回声。CDFI：血流灌注良好。
2,136,966536033984,左肾形态大小正常，包膜光整，实质回声均匀，肾中极可见一个无回声区，大小约14mmx14mm，...,肾中极可见一个无回声||区，大小约14mmx14mm，内见点状强回声。
3,481,358431899841,双肾形态大小位置正常，包膜完整，实质回声正常。左肾可见一薄壁无回声，大小约10.2×10.1...,肾可见一薄壁无回声||，大小约10.2×10.1mm。其内见强光点。CDFI：血流显示正常。...
4,1077,996774652357,左肾形态大小正常，包膜光整，实质回声均匀，肾中极可见一个无回声区，大小约27x22mm，囊壁...,肾中极可见一个无回声||区，大小约27x22mm，囊壁内见点状强回声。
...,...,...,...,...
5460,360,5ki4bojdas6ltco,右肾形态大小正常，实质回声均匀。肾内见一大小约12mmx10mm的无回声区，其内见点状强回声...,肾内见一大小约12mmx10mm的无回声||区，其内见点状强回声，变换体位时可随体位改变而移动。右
5461,560,5ki4c1fs3ixhobv,左肾形态大小正常，包膜光整，实质回声均匀，肾可见一个无回声区，大小约26mmx18mm，向窦...,肾可见一个无回声||区，大小约26mmx18mm，向窦区凸起，内见点状强回声。
5462,627,5ki4c3jss27te3u,左肾形态大小正常，包膜光整，实质回声均匀，肾内可见多个强回声，最大约6x5mm，后方伴声影。...,肾内可见一个无回声||区，大小约10x9mm，内见点状强回声。
5463,993,5ki4coyplon2adl,双肾形态大小正常范围，包膜光滑，集合系统未见明显分离。左肾内见两个无回声区，较大约18x15...,肾内见两个无回声||区，较大约18x15mm，形态欠规则，内见光带分隔。CDFI：血流灌注良好。


In [115]:
df_result.ORDER_CODE.drop_duplicates()

0    000162404317
0    385927579832
0    000367011493
0    386255498160
0    000609239134
         ...     
0    896685929416
0    317356126084
0    342801107540
0    365688365854
0    376113060845
Name: ORDER_CODE, Length: 5270, dtype: object

In [116]:
result_xd.ORDER_CODE.drop_duplicates()

0          584874661737
1          572129765294
2          966536033984
3          358431899841
4          996774652357
             ...       
5460    5ki4bojdas6ltco
5461    5ki4c1fs3ixhobv
5462    5ki4c3jss27te3u
5463    5ki4coyplon2adl
5464    5ki4d36csgirglr
Name: ORDER_CODE, Length: 5221, dtype: object

In [119]:
total = df_result.ORDER_CODE.drop_duplicates().append(result_xd.ORDER_CODE.drop_duplicates())
total1 = total.drop_duplicates(keep = False)
total1

0          635533293112
0          010570348955
0       5kfounmh3qqwpnq
0          011216447454
0       5kfp5csyyzqapfj
             ...       
5216    5ki097nteitfvfy
5239    5ki11488grpwnnl
5240    5ki114bfxmavwja
5277    5ki1cuqtz2jdcd2
5418    5ki2yee1axsb89j
Name: ORDER_CODE, Length: 395, dtype: object

In [123]:
liu = df_result.loc[df_result.ORDER_CODE.isin(total1),:]
liu.to_excel(r"C:\Users\13670\Desktop\liu.xlsx")

In [134]:
liu.drop_duplicates(subset='ORDER_CODE' )

,ORDER_CODE,文本,无回声描述,无回声外,无回声内
0,635533293112,右肾大小形态正常，包膜光整，实质回声均匀，肾中上极可见一个边清壁薄无回声区，大小约11mmx...,未见分离。CDFI：无回声区内未见血流信号。,无回声区,内未见血流信号。
0,010570348955,双肾可见多个无回声区，右侧最大约13×13mm，左侧最大约12×11mm，其内可见多个点状强...,可见多个无回声区，右侧最大约13×13mm，左侧最大约12×11mm，其内可见多个点状强回声...,无回声区，右侧最大约13×13mm，左侧最大约12×11mm，其,内可见多个点状强回声。CDFI：血流显示正常
0,5kfounmh3qqwpnq,双肾形态大小位置正常，包膜完整，实质回声正常。双肾内见无回声区，右侧大小约13×11mm，内...,内见无回声区，右侧大小约13×11mm，内见强光斑，大小约3×2mm，左侧大小约18×14m...,无回声区，右侧大小约13×11mm，内见强光斑，大小约3×2mm，左侧大小约18×14mm，...,内见钙乳回声。CDFI：血流显示正常。
0,011216447454,双肾可见无回声区，右侧大小约11×7mm，其内壁可见强回声，左侧大小约6.6×5.6mm，其...,可见无回声区，右侧大小约11×7mm，其内壁可见强回声，左侧大小约6.6×5.6mm，其内壁...,无回声区，右侧大小约11×7mm，其内壁可见强回声，左侧大小约6.6×5.6mm，其,内壁可见强回声。
0,5kfp5csyyzqapfj,双肾可见无回声区，左侧大小约12×8mm，内壁可见强回声，右侧大小约17×14mm，左肾可见...,可见无回声区，左侧大小约12×8mm，内壁可见强回声，右侧大小约17×14mm，,无回声区，左侧大小约12×8mm，,内壁可见强回声，右侧大小约17×14mm，
...,...,...,...,...,...
0,896685929416,左肾形态大小正常，包膜光整，实质回声均匀，肾实质与集合系统交界处可见一个无回声区，大小约13...,交界处可见一个无回声区，大小约13x13mm，壁薄界清，内透声好，后方回声增强。,无回声区，大小约13x13mm，壁薄界清，,内透声好，后方回声增强。
0,317356126084,左肾大小测值正常，实质回声均匀，集合系统未见明显分离及结石影。左肾盂内见10mmx10mm无...,盂内见10mmx10mm无回声，壁薄，囊内清。,无回声，壁薄，囊,内清。
0,342801107540,左肾形态大小正常，包膜光整，实质回声均匀，肾盂旁可见一个无回声区，大小约18.9mm×16....,盂旁可见一个无回声区，大小约18.9mm×16.7mm，内见点状强回声。另于,无回声区，大小约18.9mm×16.7mm，,内见点状强回声。另于
0,365688365854,左肾盂内见27.6*22.2mm无回声，壁薄，囊内清。,盂内见27.6*22.2mm无回声，壁薄，囊内清。,无回声，壁薄，囊,内清。


In [124]:
xie = result_xd.loc[result_xd.ORDER_CODE.isin(total1),:]
xie.to_excel(r"C:\Users\13670\Desktop\xie.xlsx")

In [132]:
xie

,Unnamed: 0,ORDER_CODE,desc,nei
25,6932,012657961126,肝脏大小形态正常，表面平，边缘锐，肝右叶内可见直径约 5mm强回声区，肝内管道系统走行清晰，...,肾可见大小约 14x 10mm无回声||区，其内可见点状强回声，双
34,8129,016258266721,双肾大小正常，窦区无分离。右肾实质可见一个无回声团向外突出，大小约25x21mm，边界清晰规...,肾上极可见一个无回声||团向外突出，大小约28x23mm，边界清，形态规则。CDFI：其内未...
36,9929,021567798526,左肾内可见两个类圆形无回声区，较大约（2.4）×（2.3）cm，后方回声增强。右肾内可见两个...,肾内可见两个无回声||区，较大约（5.2）×（3.0）cm，后方回声增强，内可见分隔。
51,3697,032837748372,双肾形态大小正常，皮髓质清晰，右肾内可见一个类圆形无回声区，大小约1.2×1.0cm，后方回...,肾内可见一个类圆形无回声||区，大小约1.2×1.0cm，后方回声增强，囊壁见强回声。另实质...
65,6845,041989344451,右肾囊肿并囊壁钙化右肾上极实质内可见大小约1.6*1.1cm无回声区，边界尚清，其内可见多个...,肾上极实质内可见大小约1.6*1.1cm无回声||区，边界尚清，其内可见多个强回声光斑。
...,...,...,...,...
5216,1969,5ki097nteitfvfy,肝脏大小形态正常，包膜光滑，实质回声均匀，血管走向清晰，门静脉主干无扩张。CDFI:门脉见入...,肾下极实质内探及一囊性无回声||，大小约1.36×1.24cm，壁薄，内透声好，后方回声增强；余双
5239,9769,5ki11488grpwnnl,肝脏大小正常，形态规则，内回声细密增强，分布均匀，血管走行清晰，门静脉及肝内外胆管未见扩张。...,肾可见一个类圆形无回声||区，大小约1.7x1.3cm，壁光滑，内透声好。
5240,9770,5ki114bfxmavwja,肝脏大小正常，形态规则，内回声细密增强，分布欠均匀。血管走行尚清晰，门静脉及肝内外胆管未见扩...,肾中上极可见一个类圆形无回声||区，大小约0.7x0.9cm，壁欠光滑，内透声欠佳。
5277,4385,5ki1cuqtz2jdcd2,左肾形态大小正常，包膜光整，实质内可见多个无回声区，大者约9.5 mmx9 mm，后方回声增...,肾形态大小正常，包膜光整，实质内可见多个无回声||区，大者约9.5 mmx9 mm，后方回声...


In [154]:
not_find = Recode_df1.loc[Recode_df1.ORDER_CODE.isin(xie.ORDER_CODE),:]
# not_find.to_excel(r"C:\Users\13670\Desktop\not_find.xlsx")

In [130]:
wu = pd.read_pickle(r"C:\Users\13670\Desktop\wu.pkl")
not_find1 = wu.loc[wu.ORDER_CODE.isin(liu.ORDER_CODE),:]

In [147]:
wu.loc[wu.ORDER_CODE == "061180081106","desc" ]

3400    双肾大小、形态、结构正常，集合系统未见分离；右肾实质内可见无回声\r\n，大小约3.3×2....
Name: desc, dtype: object

In [136]:
not_find1.to_excel(r"C:\Users\13670\Desktop\not_find1.xlsx")

In [76]:
test = "内可见一个无回声区，大小约27x24mm，其内可见光带分隔"
regex_find = r"((无回声.*)(内.*))"
result_ = re.search(regex_find, test)

print(result_.group(1))
print(result_.group(2))
print(result_.group(3))


无回声区，大小约27x24mm，其内可见光带分隔
无回声区，大小约27x24mm，其内可见光带分隔
无回声区，大小约27x24mm，其
内可见光带分隔


In [145]:
test = "右肾大小、形态正常，于肾实质内可见一大小约10x8mm无回声区，后方回声增强。另于其集合系统内可见多个点状强回声，后无声影，集合系统未见明显分离。"
regex2 = r"(肾|集合系统|左肾|右肾)"
result_test = re.split(regex2,test)

result_test

['',
 '右肾',
 '大小、形态正常，于',
 '肾',
 '实质内可见一大小约10x8mm无回声区，后方回声增强。另于其',
 '集合系统',
 '内可见多个点状强回声，后无声影，',
 '集合系统',
 '未见明显分离。']

In [ ]:
regex1 = r"(无回声|囊肿).*(内)(.*)(" + location + ")"

regex2 = r"(无回声|囊肿).*(内)(.*)(" + location + ")"

In [32]:
conn=pymysql.connect(host='mysql-backup.shanzhen.me',
                     user='readonly_medicine_liuqq',
                     password="r3g1^QWgV8GeYJw4",
                     database='asgard_dataplatform',
                     port= 3001,
                     charset='utf8')

#### 甲状腺钙化灶 （tag拉数据）

In [3]:
id_list = ["OBDIM0074Tag0016"]

In [4]:
tag_df = pd.DataFrame()

for items in id_list:
    id_str = "'"+ items + "'"
    tagsql = 'SELECT ORDER_CODE,TAG_ID \
    FROM t66_apollo_offline_analysis_tag \
    WHERE SOURCE_TYPE = 1 \
    AND SZ_STATUS = 1 AND TAG_ID IN ('+id_str+') '
    tag = pd.read_sql(tagsql,con=conn)
    tag_df = pd.concat([tag_df,tag],axis = 0) 
    
tag_df   

,ORDER_CODE,TAG_ID
0,000010487220,OBDIM0074Tag0016
1,385720629257,OBDIM0074Tag0016
2,385888691675,OBDIM0074Tag0016
3,000342987290,OBDIM0074Tag0016
4,386053489511,OBDIM0074Tag0016
...,...,...
8186,385078324709,OBDIM0074Tag0016
8187,385227466424,OBDIM0074Tag0016
8188,385273861381,OBDIM0074Tag0016
8189,385296391771,OBDIM0074Tag0016


In [5]:
tag_df1 = tag_df.drop_duplicates(subset = 'ORDER_CODE')
tag_df1

,ORDER_CODE,TAG_ID
0,000010487220,OBDIM0074Tag0016
1,385720629257,OBDIM0074Tag0016
2,385888691675,OBDIM0074Tag0016
3,000342987290,OBDIM0074Tag0016
4,386053489511,OBDIM0074Tag0016
...,...,...
8186,385078324709,OBDIM0074Tag0016
8187,385227466424,OBDIM0074Tag0016
8188,385273861381,OBDIM0074Tag0016
8189,385296391771,OBDIM0074Tag0016


In [10]:
Recode_df = pd.DataFrame()

for items in tag_df1.ORDER_CODE:
    ORDER_CODE_str = "'"+ items + "'"
    
    sqlstr = "SELECT DISTINCT RR.ORDER_CODE,RR2.RAW_NAME,RR.RAW_NAME,RR.RAW_RESULT AS '三级项结果',RR2.RAW_RESULT AS '小结',DR.SD_CODE \
    FROM T66_ATHENA_BASIC_RAW_RECORD AS RR LEFT JOIN T66_ATHENA_BASIC_DESCRIPTIVE_RECORD AS DR ON RR.ORDER_CODE = DR.ORDER_CODE \
    AND RR.RECORD_CODE = DR.RECORD_CODE AND DR.SD_CODE IN ('S0000016D') \
    AND RR.ORDER_CODE IN (" + ORDER_CODE_str + ")AND DR.SZ_STATUS = 1 \
    LEFT JOIN T66_ATHENA_BASIC_RAW_RECORD AS RR2 ON RR.ORDER_CODE = RR2.ORDER_CODE \
    AND RR.PARENT_RECORD_CODE = RR2.RECORD_CODE \
    WHERE \
    DR.ID IS NOT NULL \
    AND RR.SZ_STATUS = 1 \
    AND DR.SZ_STATUS = 1 \
    AND RR2.SZ_STATUS = 1 " 

    Recode = pd.read_sql(sqlstr,con=conn)
    Recode_df = pd.concat([Recode_df,Recode],axis = 0) 

Recode_df

,ORDER_CODE,RAW_NAME,RAW_NAME,三级项结果,小结,SD_CODE
0,000010487220,超声检查室,甲状腺,甲状腺右侧叶见边界尚清晰的单发结节，大小约2×2mm。CDFI：血流未见明显异常\r\n甲状...,甲状腺结节\r\n乳腺导管扩张,S0000016D
0,385720629257,彩超室,甲状腺,甲状腺双侧叶形态大小正常，峡部厚度正常，包膜完整，\r\n左侧叶探及单个低回声区，大小约9x...,甲状腺左侧叶结节伴钙化灶，建议随访\r\n甲状腺右侧叶及峡部未见明显异常\r\n肝右叶钙化灶...,S0000016D
0,385888691675,彩超室,甲状腺,甲状腺大小形态正常，峡部厚度正常，包膜光滑。实质回声均匀，左侧叶可见一个无回声区，直径约2m...,双侧颈动脉内中膜局限性增厚\r\n甲状腺左侧叶囊肿内伴钙化\r\n前列腺轻度增生伴钙化\r\...,S0000016D
0,000342987290,甲状腺超声,描述,双侧甲状腺形态大小正常，甲状腺右叶见多个低回声结节，最大约5.1mmx4.2mm，其内见点状...,甲状腺双叶结节（多发)伴钙化,S0000016D
0,386053489511,甲状腺超声,描述,甲状腺形态大小正常，边界清晰，内部回声分布不均匀。左叶见一类圆形无回声结节，大小约4x2.6...,"甲状腺双叶回声不均匀\r\n甲状腺左叶囊肿\r\n甲状腺左叶结节伴钙化,建议进一步检查",S0000016D
...,...,...,...,...,...,...
0,385078324709,超声检查室,甲状腺,甲状腺左侧叶见边界尚清晰的多个低回声，最大约4×3mm，内见点状强回声。CDFI：血流未见明...,双侧乳腺增生\r\n右侧乳腺结节待查\r\n左侧乳腺囊性结节(多发)\r\n双侧甲状腺结节(...,S0000016D
0,385227466424,超声检查室,甲状腺,1.甲状腺右侧叶见边界尚清晰的混合回声结节，大小约3mm×2mm。CDFI:结节周边及内部未...,右侧甲状腺囊实性结节\r\n左侧甲状腺结节(伴钙化)\r\n乳腺增生\r\n中度脂肪肝,S0000016D
0,385273861381,彩超室,甲状腺,双叶甲状腺形态大小正常，峡部厚度正常，包膜完整，内部回声均匀。右叶甲状腺见多个无回声大者约1...,脂肪肝\r\n肝囊肿\r\n右叶甲状腺结节（多发）伴钙化\r\n左叶甲状腺结节伴钙化\r\n...,S0000016D
0,385296391771,甲状腺超声,描述,甲状腺形态大小正常，甲状腺右叶内见一个低回声区，大小约17mm×14mm，内见点状强回声，边...,甲状腺右叶结节伴钙化,S0000016D


In [11]:
Recode_df1 = Recode_df.drop_duplicates(subset = ['ORDER_CODE','三级项结果','小结'])
Recode_df1

,ORDER_CODE,RAW_NAME,RAW_NAME,三级项结果,小结,SD_CODE
0,000010487220,超声检查室,甲状腺,甲状腺右侧叶见边界尚清晰的单发结节，大小约2×2mm。CDFI：血流未见明显异常\r\n甲状...,甲状腺结节\r\n乳腺导管扩张,S0000016D
0,385720629257,彩超室,甲状腺,甲状腺双侧叶形态大小正常，峡部厚度正常，包膜完整，\r\n左侧叶探及单个低回声区，大小约9x...,甲状腺左侧叶结节伴钙化灶，建议随访\r\n甲状腺右侧叶及峡部未见明显异常\r\n肝右叶钙化灶...,S0000016D
0,385888691675,彩超室,甲状腺,甲状腺大小形态正常，峡部厚度正常，包膜光滑。实质回声均匀，左侧叶可见一个无回声区，直径约2m...,双侧颈动脉内中膜局限性增厚\r\n甲状腺左侧叶囊肿内伴钙化\r\n前列腺轻度增生伴钙化\r\...,S0000016D
0,000342987290,甲状腺超声,描述,双侧甲状腺形态大小正常，甲状腺右叶见多个低回声结节，最大约5.1mmx4.2mm，其内见点状...,甲状腺双叶结节（多发)伴钙化,S0000016D
0,386053489511,甲状腺超声,描述,甲状腺形态大小正常，边界清晰，内部回声分布不均匀。左叶见一类圆形无回声结节，大小约4x2.6...,"甲状腺双叶回声不均匀\r\n甲状腺左叶囊肿\r\n甲状腺左叶结节伴钙化,建议进一步检查",S0000016D
...,...,...,...,...,...,...
0,385078324709,超声检查室,甲状腺,甲状腺左侧叶见边界尚清晰的多个低回声，最大约4×3mm，内见点状强回声。CDFI：血流未见明...,双侧乳腺增生\r\n右侧乳腺结节待查\r\n左侧乳腺囊性结节(多发)\r\n双侧甲状腺结节(...,S0000016D
0,385227466424,超声检查室,甲状腺,1.甲状腺右侧叶见边界尚清晰的混合回声结节，大小约3mm×2mm。CDFI:结节周边及内部未...,右侧甲状腺囊实性结节\r\n左侧甲状腺结节(伴钙化)\r\n乳腺增生\r\n中度脂肪肝,S0000016D
0,385273861381,彩超室,甲状腺,双叶甲状腺形态大小正常，峡部厚度正常，包膜完整，内部回声均匀。右叶甲状腺见多个无回声大者约1...,脂肪肝\r\n肝囊肿\r\n右叶甲状腺结节（多发）伴钙化\r\n左叶甲状腺结节伴钙化\r\n...,S0000016D
0,385296391771,甲状腺超声,描述,甲状腺形态大小正常，甲状腺右叶内见一个低回声区，大小约17mm×14mm，内见点状强回声，边...,甲状腺右叶结节伴钙化,S0000016D


In [12]:
Recode_df1.to_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\甲状腺\20201224\甲状腺钙化灶记录.xlsx")

In [40]:
Recode_df1 = pd.read_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\甲状腺\20201224\甲状腺钙化灶记录.xlsx")

In [41]:
location = pd.read_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\甲状腺\20201224\location.xlsx")
location

,location
0,肝
1,胆
2,胰
3,脾
4,肾
5,附件
6,乳腺
7,卵巢
8,前列腺
9,宫


In [ ]:
location_str = ("|".join(location.location.values))
location_str

In [ ]:
# 先用换行符号分割一次，切割完的片段中包含甲状腺&钙化
# 再用器官分割一次，在分割的片段中用“伴”&“并”，判断内外

In [82]:
import re

ex_count = 0
in_count = 0


for index,text in enumerate (Recode_df1['小结']):
    regex1 = r"[\n\r]"
    order_code = Recode_df1.ORDER_CODE[index]
    text = str(text)
    if str(text) != "" :
        result1 = re.split(regex1,text )
    else:
        text = Recode_df1.三级项结果[index]
        result1 = re.split(regex1,text)
#     print(result1)
    result_split1 = []
    for item in result1:
        if item == "" or "甲状腺" not in item:
            continue
        else:
            if "钙化" not in item:
                continue
            else:   
                result_split1.append(item)                
    if result_split1 == []:
        text = Recode_df1.三级项结果[index]
        result1 = re.split(regex1,text)
        for item_ in result1:
        if item_ == "" or "甲状腺" not in item_:
            continue
        else:
            if "钙化" not in item_:
                continue
            else:   
                result_split1.append(item_)
        
            
    location_str = ("|".join(location.location.values))
    regex2 = r"("+location_str+")"    
    result_split2 = []
    for i in result_split1:
        result2 = re.split(regex2,i )
        for item2 in result2:
            if item2 == "" or "甲状腺" not in item2:
                continue
            else:
                if "钙化" not in item2:
                    continue
                else:   
                    result_split2.append(item2)
#     print(result_split2)
#     print("================")
    
    regex3 = r"(伴钙化|并钙化)"
    for k in result_split2:
        result3 = re.findall(regex3,k )
#         print(result3)
        if result3 == []:
            ex_count +=1
        else:    
            for item3 in result3:
                in_count += 1
                   
print(ex_count)
print(in_count)

902
6642


In [83]:
print(ex_count/(ex_count+in_count)*100)
print(in_count/(ex_count+in_count)*100)

11.956521739130435
88.04347826086956


#### 甲状腺钙化灶（关键字“钙化”拉数据）

In [34]:
# 描述中包含钙化
Recode_df2 = pd.DataFrame()

sqlstr2 = "SELECT DISTINCT RR.ORDER_CODE,RR2.RAW_NAME,RR.RAW_NAME,RR.RAW_RESULT AS '三级项结果',RR2.RAW_RESULT AS '小结',DR.SD_CODE \
FROM T66_ATHENA_BASIC_RAW_RECORD AS RR LEFT JOIN T66_ATHENA_BASIC_DESCRIPTIVE_RECORD AS DR ON RR.ORDER_CODE = DR.ORDER_CODE \
AND RR.RECORD_CODE = DR.RECORD_CODE AND DR.SD_CODE IN ('S0000016D') AND DR.SZ_STATUS = 1 \
LEFT JOIN T66_ATHENA_BASIC_RAW_RECORD AS RR2 ON RR.ORDER_CODE = RR2.ORDER_CODE \
AND RR.PARENT_RECORD_CODE = RR2.RECORD_CODE \
WHERE \
DR.ID IS NOT NULL \
AND RR.SZ_STATUS = 1 \
AND DR.SZ_STATUS = 1 \
AND RR2.SZ_STATUS = 1 \
AND (RR.RAW_RESULT LIKE '%钙\s?化%')" 

Recode2 = pd.read_sql(sqlstr2,con=conn)
Recode_df2 = pd.concat([Recode_df2,Recode2],axis = 0) 

Recode_df2

,ORDER_CODE,RAW_NAME,RAW_NAME,三级项结果,小结,SD_CODE


In [ ]:
# 描述中包含钙化
Recode_df2 = pd.DataFrame()

sqlstr2 = "SELECT DISTINCT RR.ORDER_CODE,RR2.RAW_NAME,RR.RAW_NAME,RR.RAW_RESULT AS '三级项结果',RR2.RAW_RESULT AS '小结',DR.SD_CODE \
FROM T66_ATHENA_BASIC_RAW_RECORD AS RR LEFT JOIN T66_ATHENA_BASIC_DESCRIPTIVE_RECORD AS DR ON RR.ORDER_CODE = DR.ORDER_CODE \
AND RR.RECORD_CODE = DR.RECORD_CODE AND DR.SD_CODE IN ('S0000016D') AND DR.SZ_STATUS = 1 \
LEFT JOIN T66_ATHENA_BASIC_RAW_RECORD AS RR2 ON RR.ORDER_CODE = RR2.ORDER_CODE \
AND RR.PARENT_RECORD_CODE = RR2.RECORD_CODE \
WHERE \
DR.ID IS NOT NULL \
AND RR.SZ_STATUS = 1 \
AND DR.SZ_STATUS = 1 \
AND RR2.SZ_STATUS = 1 \
AND (RR.RAW_RESULT LIKE '%钙化%')" 

Recode2 = pd.read_sql(sqlstr2,con=conn)
Recode_df2 = pd.concat([Recode_df2,Recode2],axis = 0)

In [36]:
Recode_df2

,ORDER_CODE,RAW_NAME,RAW_NAME,三级项结果,小结,SD_CODE
0,299025478169,超声检查室,甲状腺,甲状腺左侧叶见边界尚清晰的低回声结节伴钙化，大小约2.4×2.2mm。,中度脂肪肝\r\n左肾囊肿\r\n左侧甲状腺结节(伴钙化)\r\n颈动脉内中膜增厚,S0000016D
1,100063212006,甲状腺超声,甲状腺超声,左叶甲状腺探及约（0.2）x（0.2）cm低回声结节。形态规则，界欠清，回声欠均匀\r\n，...,左叶甲状腺结节弹性超声评分：2级，建议定期复查或专科诊治。,S0000016D
2,001405124627,甲状腺超声,描述,甲状腺大小正常，被膜光滑，实质回声均匀。CDFI：腺体内血流分布正常。\r\n于甲状腺左叶可...,甲状腺左叶结节伴钙化\r\n甲状腺峡部结节\r\n右侧颈部淋巴结肿大,S0000016D
3,001603891216,超声检查室,甲状腺,甲状腺右侧叶见边界尚清晰的多发低回声结节，最大约5×3mm，部分可见钙化。CDFI：周边可见...,宫内节育器位置正常\r\n甲状腺结节(右侧多发，部分伴钙化)\r\n胆囊息肉,S0000016D
4,010410644257,超声检查室,甲状腺,双侧甲状腺见边界尚清晰的多发结节，左侧最大约8×5mm，右侧最大约15x7mm，内均可见点状...,甲状腺结节(双侧多发，建议复查)\r\n前列腺增生,S0000016D
...,...,...,...,...,...,...
1071,5kizjay6mkoojef,甲状腺超声,描述,甲状腺形态大小正常，包膜完整，实质回声稍减低、欠均质，并见条索状强回声，另于右叶内可见一斑点...,甲状腺双叶回声欠均匀\r\n甲状腺右叶钙化灶,S0000016D
1072,5kizuedkzufs7nc,甲状腺超声,描述,甲状腺右侧叶可见12.3x8.3mm囊实性结节，边界清晰，其内可见点状钙化灶，左侧叶可见12...,甲状腺右侧叶结节伴钙化，建议超声造影\r\n甲状腺左侧叶结节，考虑结甲,S0000016D
1073,5kj02pxgomsww7q,甲状腺超声,描述,甲状腺双侧叶可见多发囊实性结节，内呈海绵状，左侧叶较大6.7x6.4mm，右侧叶较大9.2x...,甲状腺双侧叶结节，考虑结甲伴部分钙化,S0000016D
1074,5kj1m7fojnpvyha,甲状腺超声,描述,甲状腺体积偏小，内部回声不均匀，双侧叶可见2.4mm囊肿，右侧叶可见3.8x2.7mm低回声...,甲状腺双侧叶囊肿\r\n甲状腺右侧叶结节，考虑结甲伴钙化\r\n甲状腺体积偏小，请结合甲功,S0000016D


In [38]:
# 结论中包含钙化

Recode_df3 = pd.DataFrame()

sqlstr3 = "SELECT DISTINCT RR.ORDER_CODE,RR2.RAW_NAME,RR.RAW_NAME,RR.RAW_RESULT AS '三级项结果',RR2.RAW_RESULT AS '小结',DR.SD_CODE \
FROM T66_ATHENA_BASIC_RAW_RECORD AS RR LEFT JOIN T66_ATHENA_BASIC_DESCRIPTIVE_RECORD AS DR ON RR.ORDER_CODE = DR.ORDER_CODE \
AND RR.RECORD_CODE = DR.RECORD_CODE AND DR.SD_CODE IN ('S0000016D') AND DR.SZ_STATUS = 1 \
LEFT JOIN T66_ATHENA_BASIC_RAW_RECORD AS RR2 ON RR.ORDER_CODE = RR2.ORDER_CODE \
AND RR.PARENT_RECORD_CODE = RR2.RECORD_CODE \
WHERE \
DR.ID IS NOT NULL \
AND RR.SZ_STATUS = 1 \
AND DR.SZ_STATUS = 1 \
AND RR2.SZ_STATUS = 1 \
AND (RR2.RAW_RESULT LIKE '%钙化')" 

Recode3 = pd.read_sql(sqlstr3,con=conn)
Recode_df3 = pd.concat([Recode_df3,Recode3],axis = 0) 

Recode_df3

,ORDER_CODE,RAW_NAME,RAW_NAME,三级项结果,小结,SD_CODE
0,943274800632,甲状腺超声,描述,甲状腺形态大小正常，双侧叶内均可见多个低回声区，较大位于右侧叶，大小约12.5x9.3mm，...,甲状腺双叶结节伴钙化,S0000016D
1,310714435753,彩超室,甲状腺,双叶甲状腺切面形态失常，体积增大，包膜欠清晰，内部回声减弱，分布不均匀。CDFI：未见明显异...,肝囊肿，建议复查\r\n肝内低回声，建议复查\r\n胆囊结晶\r\n脾肿大，建议进一步检查\...,S0000016D
2,335121913442,甲状腺超声,描述,双侧甲状腺大小形态正常，包膜光整，实质回声均匀，右侧内见两个低回声，较大的约0.8x0.6c...,甲状腺右侧多发结节伴右侧钙化,S0000016D
3,124589255698,超声检查,甲状腺超声,甲状腺彩超：甲状腺形态大小正常，回声均匀\r\n，未见占位性病变，腺体血流未见异常。,甲状腺彩超:甲状腺未见异常\r\n腹部彩超:脂肪肝（轻度）\r\n胰腺显示不清晰（受肠气影响...,S0000016D
4,207247111540,超声检查,甲状腺超声,甲状腺彩超：甲状腺形态大小正常，回声欠均\r\n匀，未见占位性病变，腺体血流未见异常。,脂肪肝（轻度）\r\n肝囊肿\r\n甲状腺回声欠均匀\r\n子宫萎缩伴肌壁钙化,S0000016D
...,...,...,...,...,...,...
2380,5kizj9ccferymnn,甲状腺超声,描述,双侧甲状腺形态大小正常，甲状腺右叶内见一个低回声结节，大小约11mm×10mm，边界清晰，形...,甲状腺双叶结节伴钙化,S0000016D
2381,5kj02pxgomsww7q,甲状腺超声,描述,甲状腺双侧叶可见多发囊实性结节，内呈海绵状，左侧叶较大6.7x6.4mm，右侧叶较大9.2x...,甲状腺双侧叶结节，考虑结甲伴部分钙化,S0000016D
2382,5kj0s2mumvhsj8t,甲状腺超声,描述,"甲状腺形态饱满，右叶前后径1.7cm，左叶前后径2.0cm,右叶可见数个不均回声结节及低回声...",甲状腺双双叶回声不均匀\r\n甲状腺双叶结节\r\n甲状腺双叶结节伴钙化,S0000016D
2383,5kj1dqhr3h4jozs,彩超室,甲状腺,双叶甲状腺形态大小正常，峡部厚度正常，包膜完整，内部回声均匀。右叶见多个低回声，之一大小约5...,双叶甲状腺多发结节\r\n双侧颈动脉斑块\r\n脂肪肝\r\n双肾囊肿\r\n胆、胰、脾目前...,S0000016D


In [39]:
# 结论中包含钙化

Recode_df3 = pd.DataFrame()

sqlstr3 = "SELECT DISTINCT RR.ORDER_CODE,RR2.RAW_NAME,RR.RAW_NAME,RR.RAW_RESULT AS '三级项结果',RR2.RAW_RESULT AS '小结',DR.SD_CODE \
FROM T66_ATHENA_BASIC_RAW_RECORD AS RR LEFT JOIN T66_ATHENA_BASIC_DESCRIPTIVE_RECORD AS DR ON RR.ORDER_CODE = DR.ORDER_CODE \
AND RR.RECORD_CODE = DR.RECORD_CODE AND DR.SD_CODE IN ('S0000016D') AND DR.SZ_STATUS = 1 \
LEFT JOIN T66_ATHENA_BASIC_RAW_RECORD AS RR2 ON RR.ORDER_CODE = RR2.ORDER_CODE \
AND RR.PARENT_RECORD_CODE = RR2.RECORD_CODE \
WHERE \
DR.ID IS NOT NULL \
AND RR.SZ_STATUS = 1 \
AND DR.SZ_STATUS = 1 \
AND RR2.SZ_STATUS = 1 \
AND (RR2.RAW_RESULT LIKE '%钙化%')" 

Recode3 = pd.read_sql(sqlstr3,con=conn)
Recode_df3 = pd.concat([Recode_df3,Recode3],axis = 0) 

Recode_df3

,ORDER_CODE,RAW_NAME,RAW_NAME,三级项结果,小结,SD_CODE
0,055851700663,甲状腺超声,描述,甲状腺形态大小正常，甲状腺左叶可见两个结节，较大约11.3x9.7mm，内可见点状强回声，边...,"甲状腺左叶结节伴钙化,建议进一步检查",S0000016D
1,323903352666,超声检查室,甲状腺,未见明显异常,轻度脂肪肝\r\n前列腺钙化灶,S0000016D
2,584874661737,彩超室,甲状腺,双叶甲状腺形态大小正常，峡部厚度正常，包膜完整，内部回声欠均匀。双叶内见混合回声区，较大的位...,*\r\n1.右肾囊肿\r\n2.肝囊肿\r\n3.胆囊壁稍毛糙\r\n4.右肾结石\r\n...,S0000016D
3,546511029446,彩超室,甲状腺,双叶甲状腺形态大小正常，峡部厚度正常，包膜完整，内部回声均匀。右叶甲状腺内见多个低回声大者约...,肝内钙化灶\r\n右叶甲状腺结节（多发）\r\n左叶甲状腺胶质囊肿\r\n胆、胰、脾、肾目前...,S0000016D
4,508983416958,彩超室,甲状腺,双叶甲状腺形态大小正常，峡部厚度正常，包膜完整，内部回声均匀。CDFI：未见明显异常血流信号。,胆囊壁赘生物\r\n前列腺增生伴钙化\r\n双叶甲状腺未见明显异常\r\n肝、胰、脾、肾目前...,S0000016D
...,...,...,...,...,...,...
29421,5kj2etxuyokqvig,甲状腺超声,描述,双侧甲状腺形态大小正常，包膜光整，内部回声欠均匀。甲状腺右叶内见一个低回声区，大小约2mm×...,甲状腺右叶结节伴钙化\r\n甲状腺左叶结节\r\n甲状腺双叶回声欠均匀,S0000016D
29422,5kj2eutchnjnc09,甲状腺超声,描述,甲状腺形态大小正常，包膜光整，甲状腺右叶内见多个低回声区，大者约1.8mm×3.8mm，边界...,甲状腺双叶结节（多发）伴钙化,S0000016D
29423,5kj2ewhexur9bb2,甲状腺超声,描述,甲状腺形态大小正常，包膜完整，于右叶可见多个囊实性结节，较大者约7.8x6mm，界清，内见点...,甲状腺右叶结节（多发）伴钙化\r\n甲状腺双叶囊肿并胶质形成,S0000016D
29424,5kj2hkc6mpnimyy,彩超室,甲状腺,双叶甲状腺形态大小正常，峡部厚度正常，包膜完整，内部回声均匀。右叶甲状腺内见无回声大小约3x...,前列腺钙化灶\r\n右叶甲状腺胶质囊肿\r\n肝、胆、胰、脾、肾目前未见明显占位性病变\r\...,S0000016D


#### 肾钙化灶(tag拉数据)

In [16]:
id_list_ = ["OBDIM0006Tag0054"]

tag_df_ = pd.DataFrame()

for items in id_list_:
    id_str = "'"+ items + "'"
    tagsql = 'SELECT ORDER_CODE,TAG_ID \
    FROM t66_apollo_offline_analysis_tag \
    WHERE SOURCE_TYPE = 1 \
    AND SZ_STATUS = 1 AND TAG_ID IN ('+id_str+') '
    tag = pd.read_sql(tagsql,con=conn)
    tag_df_ = pd.concat([tag_df_,tag],axis = 0) 
    
tag_df_  

,ORDER_CODE,TAG_ID
0,000018223733,OBDIM0006Tag0054
1,000191430385,OBDIM0006Tag0054
2,385843626767,OBDIM0006Tag0054
3,385864648594,OBDIM0006Tag0054
4,000370337377,OBDIM0006Tag0054
...,...,...
8230,384679308024,OBDIM0006Tag0054
8231,384755454131,OBDIM0006Tag0054
8232,385232314716,OBDIM0006Tag0054
8233,385263401598,OBDIM0006Tag0054


In [17]:
tag_df1_ = tag_df_.drop_duplicates(subset = 'ORDER_CODE')
tag_df1_

,ORDER_CODE,TAG_ID
0,000018223733,OBDIM0006Tag0054
1,000191430385,OBDIM0006Tag0054
2,385843626767,OBDIM0006Tag0054
3,385864648594,OBDIM0006Tag0054
4,000370337377,OBDIM0006Tag0054
...,...,...
8230,384679308024,OBDIM0006Tag0054
8231,384755454131,OBDIM0006Tag0054
8232,385232314716,OBDIM0006Tag0054
8233,385263401598,OBDIM0006Tag0054


In [20]:
Recode_df_ = pd.DataFrame()

for items in tag_df1_.ORDER_CODE:
    ORDER_CODE_str = "'"+ items + "'"
    
    sqlstr = "SELECT DISTINCT RR.ORDER_CODE,RR2.RAW_NAME,RR.RAW_NAME,RR.RAW_RESULT AS '三级项结果',RR2.RAW_RESULT AS '小结',DR.SD_CODE \
    FROM T66_ATHENA_BASIC_RAW_RECORD AS RR LEFT JOIN T66_ATHENA_BASIC_DESCRIPTIVE_RECORD AS DR ON RR.ORDER_CODE = DR.ORDER_CODE \
    AND RR.RECORD_CODE = DR.RECORD_CODE AND DR.SD_CODE IN ('S0000005D','S0000006D','S0000007D') \
    AND RR.ORDER_CODE IN (" + ORDER_CODE_str + ")AND DR.SZ_STATUS = 1 \
    LEFT JOIN T66_ATHENA_BASIC_RAW_RECORD AS RR2 ON RR.ORDER_CODE = RR2.ORDER_CODE \
    AND RR.PARENT_RECORD_CODE = RR2.RECORD_CODE \
    WHERE \
    DR.ID IS NOT NULL \
    AND RR.SZ_STATUS = 1 \
    AND DR.SZ_STATUS = 1 \
    AND RR2.SZ_STATUS = 1 " 

    Recode_ = pd.read_sql(sqlstr,con=conn)
    Recode_df_ = pd.concat([Recode_df_,Recode_],axis = 0) 

Recode_df_

,ORDER_CODE,RAW_NAME,RAW_NAME,三级项结果,小结,SD_CODE
0,000018223733,超声检查室,双肾,双肾可见多个强回声，最大约7×5mm,轻度脂肪肝\r\n肾钙化灶\r\n前列腺钙化灶,S0000005D
0,000191430385,彩超室,双肾,双肾形态大小正常范围，包膜光滑，集合系统未见明显分离。双肾内见细点状及短条状中高回声。CDF...,双肾钙化灶\r\n肝、胆、胰、脾目前未见明显占位性病变\r\n甲状腺未见明显异常\r\n前列...,S0000005D
0,385843626767,彩超室,双肾,左肾实质内见一强回声，直径约5mm，右肾形态大小正常范围，包膜光滑，集合系统未见明显分离。C...,左肾钙化灶\r\n肝、胆、胰、脾、右肾目前未见明显占位性病变,S0000005D
0,385864648594,超声检查室,双肾,双肾形态大小位置正常，包膜完整，实质回声正常。右肾可见一个强回声，大小约4×3mm。CDFI...,右肾钙化灶,S0000005D
0,000370337377,腹部超声,左肾,左肾形态大小正常，包膜光整，实质回声均匀，内可见强回声，大小约4.0mmx3.1mm，后无明...,肝:脂肪肝（轻度）\r\n左肾:左肾内钙化灶\r\n胆、胰、脾、右肾未发现明显异常,S0000006D
...,...,...,...,...,...,...
0,385232314716,肝胆胰脾肾超声,左肾,左肾形态大小正常，包膜光整，实质回声均匀，肾上极可见一个无回声区，大小约18mmx15mmx...,肝:肝囊肿（多发）\r\n胆:胆囊息肉（多发）\r\n左肾:多发左肾囊肿伴钙化\r\n左肾内...,S0000006D
1,385232314716,肝胆胰脾肾超声,右肾,右肾形态大小正常，包膜光整，实质回声均匀，肾内可见多个强回声，最大者约5mmx4mm，后方伴...,肝:肝囊肿（多发）\r\n胆:胆囊息肉（多发）\r\n左肾:多发左肾囊肿伴钙化\r\n左肾内...,S0000007D
0,385263401598,彩超室,双肾,双肾形态大小正常范围，包膜光滑，集合系统未见明显分离。左肾内见短条状中高回声。CDFI：血流...,肝内脂肪浸润\r\n胆囊息肉样病变\r\n左肾钙化灶可能\r\n胰、脾、右肾目前未见明显占位...,S0000005D
0,385634629857,腹部超声,左肾,左肾大小、形态正常，包膜光滑，肾实质回声均匀，集合系统未见明显分离。,肝:肝内钙化灶 胆、胰、脾、右肾、左肾未发现明显异常,S0000006D


In [21]:
Recode_df1_ = Recode_df_.drop_duplicates(subset = ['ORDER_CODE','三级项结果','小结'])
Recode_df1_

,ORDER_CODE,RAW_NAME,RAW_NAME,三级项结果,小结,SD_CODE
0,000018223733,超声检查室,双肾,双肾可见多个强回声，最大约7×5mm,轻度脂肪肝\r\n肾钙化灶\r\n前列腺钙化灶,S0000005D
0,000191430385,彩超室,双肾,双肾形态大小正常范围，包膜光滑，集合系统未见明显分离。双肾内见细点状及短条状中高回声。CDF...,双肾钙化灶\r\n肝、胆、胰、脾目前未见明显占位性病变\r\n甲状腺未见明显异常\r\n前列...,S0000005D
0,385843626767,彩超室,双肾,左肾实质内见一强回声，直径约5mm，右肾形态大小正常范围，包膜光滑，集合系统未见明显分离。C...,左肾钙化灶\r\n肝、胆、胰、脾、右肾目前未见明显占位性病变,S0000005D
0,385864648594,超声检查室,双肾,双肾形态大小位置正常，包膜完整，实质回声正常。右肾可见一个强回声，大小约4×3mm。CDFI...,右肾钙化灶,S0000005D
0,000370337377,腹部超声,左肾,左肾形态大小正常，包膜光整，实质回声均匀，内可见强回声，大小约4.0mmx3.1mm，后无明...,肝:脂肪肝（轻度）\r\n左肾:左肾内钙化灶\r\n胆、胰、脾、右肾未发现明显异常,S0000006D
...,...,...,...,...,...,...
0,385232314716,肝胆胰脾肾超声,左肾,左肾形态大小正常，包膜光整，实质回声均匀，肾上极可见一个无回声区，大小约18mmx15mmx...,肝:肝囊肿（多发）\r\n胆:胆囊息肉（多发）\r\n左肾:多发左肾囊肿伴钙化\r\n左肾内...,S0000006D
1,385232314716,肝胆胰脾肾超声,右肾,右肾形态大小正常，包膜光整，实质回声均匀，肾内可见多个强回声，最大者约5mmx4mm，后方伴...,肝:肝囊肿（多发）\r\n胆:胆囊息肉（多发）\r\n左肾:多发左肾囊肿伴钙化\r\n左肾内...,S0000007D
0,385263401598,彩超室,双肾,双肾形态大小正常范围，包膜光滑，集合系统未见明显分离。左肾内见短条状中高回声。CDFI：血流...,肝内脂肪浸润\r\n胆囊息肉样病变\r\n左肾钙化灶可能\r\n胰、脾、右肾目前未见明显占位...,S0000005D
0,385634629857,腹部超声,左肾,左肾大小、形态正常，包膜光滑，肾实质回声均匀，集合系统未见明显分离。,肝:肝内钙化灶 胆、胰、脾、右肾、左肾未发现明显异常,S0000006D


In [22]:
Recode_df1_.to_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\肾脏\20201224_\肾脏钙化灶记录.xlsx")

In [84]:
Recode_df1_ = pd.read_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\肾脏\20201224_\肾脏钙化灶记录.xlsx")

In [95]:
location1 = pd.read_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\肾脏\20201224\location.xlsx")

In [99]:
import re

ex_count1 = 0
in_count1 = 0

for index,text in enumerate (Recode_df1_['小结']):
    regex1 = r"[\n\r]"
    order_code = Recode_df1_.ORDER_CODE[index]
    text = str(text)
    if str(text) != "" :
        result1 = re.split(regex1,text )
    else:
        text = Recode_df1_.三级项结果[index]
        result1 = re.split(regex1,text)
#     print(result1)
    result_split1 = []
    for item in result1:
        if item == "" or ("肾" not in item):
            continue
        else:
            if "钙化" not in item:
                continue
            else:   
                result_split1.append(item)
#     print(result_split1)

    
    location_str = ("|".join(location1.location.values))
    regex2 = r"("+location_str+")"
    result_split2 = []
    for i in result_split1:
        result2 = re.split(regex2,i )
#         print(result2)
        for item2 in result2:
            if item2 == "" or ("肾" not in item2):
                continue
            else:
                if "钙化" not in item2:
                    continue
                else:   
                    result_split2.append(item2)
#     print(result_split2)
#     print("================")
    
    regex3 = r"(伴钙化|并钙化)"
    for k in result_split2:
        result3 = re.findall(regex3,k )
        print(result3)
        if result3 == []:
            ex_count1 +=1
        else:    
            for item3 in result3:
                in_count1 += 1
                   
print(ex_count1)
print(in_count1)

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['伴钙化']
['伴钙化']
[]
[]
[]
[]
[]
[]
[]
[]
[]
['伴钙化']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['伴钙化']
['伴钙化']
[]
['伴钙化']
[]
['伴钙化']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['伴钙化', '伴钙化']
['伴钙化', '伴钙化']
[]
[]
[]
[]
[]
[]
[]
['伴钙化']
['伴钙化']
[]
[]
[]
[]
['伴钙化']
['伴钙化']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['伴钙化']
['伴钙化']
['伴钙化']
[]
[]
['伴钙化']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['伴钙化']
['伴钙化']
[]
[]
[]
['伴钙化']
[]
[]
[]
[]
[]
[]
[]
['伴钙化']
['伴钙化']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['伴钙化']
['伴钙化']
['伴钙化']
[]
[]
[]
[]
[]
[]
[]
[]
['伴钙化']
['伴钙化']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['伴钙化']
[]
[]
['伴钙化']
['伴钙化']
[]
[]
[]
[]
[]
[]
['伴钙化']
['伴钙化']
[]
[]
[]
['伴钙化']
['伴钙化']
['伴钙化']
['伴钙化']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['伴钙化']
['伴钙化']
['伴钙化']
[]
[]
[]
['伴钙化']
['伴钙化']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['伴钙化']
['伴钙化']
[]
[]
['伴钙化']
['伴钙化']
[]
[]
['伴钙化']
['伴钙化']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['伴钙化']
[]
[]
['

[]
[]
['伴钙化']
['伴钙化']
[]
['伴钙化']
['伴钙化']
[]
[]
[]
[]
['伴钙化']
['伴钙化']
['伴钙化']
['伴钙化']
['伴钙化']
['伴钙化']
[]
[]
['伴钙化']
['伴钙化']
['伴钙化']
['伴钙化']
[]
[]
['伴钙化']
['伴钙化']
['伴钙化']
['伴钙化']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['伴钙化']
['伴钙化']
[]
[]
['伴钙化']
['伴钙化']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['伴钙化']
['伴钙化']
['伴钙化']
[]
[]
['伴钙化']
['伴钙化']
[]
[]
['伴钙化']
['伴钙化']
['伴钙化']
['伴钙化']
['伴钙化']
['伴钙化']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['伴钙化']
['伴钙化']
['伴钙化']
['伴钙化']
['伴钙化']
['伴钙化']
['伴钙化']
['伴钙化']
[]
[]
[]
[]
[]
[]
['伴钙化']
['伴钙化']
[]
[]
[]
['伴钙化']
['伴钙化']
[]
[]
[]
[]
['伴钙化']
['伴钙化']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['伴钙化']
['伴钙化']
[]
['伴钙化']
['伴钙化']
[]
[]
[]
['伴钙化']
['伴钙化']
['伴钙化']
['伴钙化']
['伴钙化']
[]
[]
[]
[]
[]
[]
['伴钙化']
['伴钙化']
['伴钙化']
['伴钙化']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['伴钙化']
['伴钙化']
[]
['伴钙化']
['伴钙化']
[]
[]
['伴钙化']
['伴钙化']
[]
[]
[]
[]
['伴钙化']
['伴钙化']
[]
[]
[]
[]
[]
[]
[]
[]
['伴钙化']
['伴钙化']
['伴钙化']
['伴钙化']
[]
[]
[]
[]
[]
['并钙化']
['并钙化']
[]
[]
[]
[]
['伴钙化']
['伴钙化']
[]
[]
[]
[]
[]
[]

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['伴钙化']
['伴钙化']
['伴钙化']
['伴钙化']
[]
[]
[]
[]
[]
['伴钙化']
['伴钙化']
[]
[]
[]
[]
['伴钙化']
['伴钙化']
['伴钙化']
['伴钙化']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['伴钙化']
[]
['伴钙化']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['伴钙化']
['伴钙化']
['伴钙化']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['伴钙化']
['伴钙化']
[]
[]
[]
[]
[]
[]
[]
['伴钙化']
['伴钙化']
[]
[]
['伴钙化']
['伴钙化']
['伴钙化']
['伴钙化']
['伴钙化']
['伴钙化']
[]
[]
['伴钙化']
['伴钙化']
[]
[]
[]
[]
[]
[]
[]
[]
['伴钙化']
['伴钙化']
[]
['伴钙化']
['伴钙化']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['伴钙化']
['伴钙化']
['伴钙化']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['伴钙化']
['伴钙化']
[]
[]
[]
[]
[]
[]
['伴钙化']
['伴钙化']
['伴钙化']
['伴钙化']
[]
[]
[]
['伴钙化']
['伴钙化']
['伴钙化']
['伴钙化']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['伴钙化']
['伴钙化']
[]
[]
[]
[]
[]
[]
['伴钙化']
['伴钙化']
[]
[]
['伴钙化']
['伴钙化']
[]
[]
['伴钙化']
['伴钙化']
['伴钙化']
['伴钙化']
[]
[]
[]
[]
[]
[]
['伴钙化']
['伴钙化']
[]
['伴钙化']
[]
[]
[]
[]
['伴钙化']
['伴钙化

['伴钙化']
[]
['伴钙化']
['伴钙化']
['伴钙化']
['伴钙化']
['伴钙化']
['伴钙化']
[]
[]
['伴钙化']
['伴钙化']
['伴钙化']
['伴钙化']
[]
[]
[]
[]
['伴钙化']
['伴钙化']
[]
[]
[]
[]
[]
['伴钙化']
['伴钙化']
['伴钙化']
['伴钙化']
['伴钙化']
['伴钙化']
[]
[]
['伴钙化']
['伴钙化']
['伴钙化']
['伴钙化']
['伴钙化']
['伴钙化']
[]
[]
[]
[]
['伴钙化']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['伴钙化']
['伴钙化']
[]
[]
[]
[]
[]
[]
[]
[]
['伴钙化']
['伴钙化']
['伴钙化']
['伴钙化']
[]
[]
[]
['伴钙化']
['伴钙化']
[]
[]
[]
[]
[]
[]
['伴钙化']
['伴钙化']
[]
[]
[]
[]
['伴钙化']
['伴钙化']
['伴钙化']
['伴钙化']
[]
['伴钙化']
['伴钙化']
[]
[]
[]
[]
[]
[]
[]
['伴钙化']
['伴钙化']
[]
[]
['伴钙化']
['伴钙化']
['伴钙化']
['伴钙化']
['伴钙化']
['伴钙化']
['伴钙化']
['伴钙化']
[]
[]
[]
[]
[]
[]
[]
[]
['伴钙化']
['伴钙化']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['伴钙化']
['伴钙化']
['伴钙化']
['伴钙化']
[]
[]
['伴钙化']
['伴钙化']
[]
[]
[]
[]
[]
[]
['伴钙化']
['伴钙化']
['伴钙化']
['伴钙化']
['伴钙化']
['伴钙化']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['伴钙化']
['伴钙化']
[]
[]
[]
[]
[]
[]
[]
['伴钙化']
['伴钙化']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['伴钙化']
['伴钙化']
['伴钙化']
[]
['伴钙化']
[]
[]
[]
[]
['伴钙化'

In [101]:
a = round(ex_count1/(ex_count1+in_count1)*100,2)
b = round(in_count1/(ex_count1+in_count1)*100,2)

In [103]:
pd.DataFrame({"位置":["内钙化","外钙化"],"数量":[in_count1,ex_count1],"占比(%)":[b,a]})

,位置,数量,占比(%)
0,内钙化,3315,29.13
1,外钙化,8066,70.87
